```{contents}
```
## **Consensus in LangGraph**

In LangGraph, **Consensus** is a **multi-agent coordination pattern** where multiple autonomous agents independently analyze a problem and then converge on a single, higher-quality decision through structured agreement mechanisms.
It is essential for **robust reasoning, error reduction, and reliability** in production LLM systems.

---

### **1. Motivation**

Single-agent LLM systems suffer from:

| Limitation     | Impact                  |
| -------------- | ----------------------- |
| Hallucinations | Incorrect outputs       |
| Bias           | Skewed decisions        |
| Overconfidence | Unchecked errors        |
| Instability    | High variance responses |

**Consensus** mitigates these issues by introducing **redundant intelligence** and **cross-verification**.

---

### **2. Conceptual Model**

```
           ┌────────── Agent A ──────────┐
Input ───▶ │                             │
           ├────────── Agent B ──────────┤ ──▶ Aggregation ──▶ Final Decision
           │                             │
           └────────── Agent C ──────────┘
```

Each agent works **independently**.
A **consensus node** merges their outputs.

---

### **3. Core Components in LangGraph**

| Component      | Role                      |
| -------------- | ------------------------- |
| Agents         | Independent solvers       |
| Shared State   | Message & result exchange |
| Supervisor     | Orchestrates execution    |
| Consensus Node | Resolves agreement        |
| Reducer        | Merges agent outputs      |
| Control Logic  | Determines convergence    |

---

### **4. Workflow in LangGraph**

1. **Fan-out**: Supervisor sends the same task to multiple agents
2. **Parallel reasoning**: Agents produce independent results
3. **Collection**: Results stored in state
4. **Aggregation**: Consensus node evaluates responses
5. **Decision**: Final answer chosen
6. **Optional reflection loop**: Repeat if disagreement is high

---

### **5. State Design**

```python
class State(TypedDict):
    task: str
    responses: list[str]
    final: str
```

---

### **6. Minimal Working Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    task: str
    responses: list[str]
    final: str

def agent_a(state):
    return {"responses": state["responses"] + ["Answer from A"]}

def agent_b(state):
    return {"responses": state["responses"] + ["Answer from B"]}

def agent_c(state):
    return {"responses": state["responses"] + ["Answer from C"]}

def consensus(state):
    # Simple voting / heuristic
    state["final"] = max(set(state["responses"]), key=state["responses"].count)
    return state

builder = StateGraph(State)

builder.add_node("A", agent_a)
builder.add_node("B", agent_b)
builder.add_node("C", agent_c)
builder.add_node("Consensus", consensus)

builder.set_entry_point("A")
builder.add_edge("A", "B")
builder.add_edge("B", "C")
builder.add_edge("C", "Consensus")
builder.add_edge("Consensus", END)

graph = builder.compile()
```

---

### **7. Advanced Consensus Strategies**

| Strategy           | Description                       |
| ------------------ | --------------------------------- |
| Majority Vote      | Most common answer wins           |
| Weighted Voting    | Expert agents have more influence |
| Confidence Scoring | Weight by model confidence        |
| Peer Review        | Agents critique each other        |
| Deliberation Loop  | Debate until convergence          |
| Verifier Agent     | Final fact-checking pass          |
| Self-Consistency   | Sample many and select best       |

---

### **8. Production-Grade Enhancements**

| Feature                | Purpose                    |
| ---------------------- | -------------------------- |
| Disagreement Threshold | Trigger re-run             |
| Agent Diversity        | Different models / prompts |
| Traceability           | Full decision audit        |
| Timeouts               | Prevent deadlocks          |
| Human Override         | Compliance & safety        |

---

### **9. Why Consensus Improves Systems**

| Without Consensus       | With Consensus           |
| ----------------------- | ------------------------ |
| High hallucination risk | Strong error suppression |
| Unstable outputs        | Stable decisions         |
| Single point of failure | Fault tolerant           |
| Low trust               | High reliability         |

---

### **10. When to Use Consensus**

* High-risk decisions (finance, healthcare, compliance)
* Research synthesis
* Autonomous agents
* Complex reasoning pipelines
* Strategic planning systems

---

### **Key Insight**

> **Consensus transforms LLMs from stochastic generators into reliable decision systems.**



### Demonstration

In [1]:
# Demonstration of Consensus Pattern in LangGraph (single executable cell)

from langgraph.graph import StateGraph, END
from typing import TypedDict, List

# 1. Define shared state
class State(TypedDict):
    task: str
    responses: List[str]
    final: str

# 2. Define agents
def agent_a(state: State):
    return {"responses": state["responses"] + ["Paris is the capital of France."]}

def agent_b(state: State):
    return {"responses": state["responses"] + ["The capital of France is Paris."]}

def agent_c(state: State):
    return {"responses": state["responses"] + ["France's capital city is Paris."]}

# 3. Consensus node
def consensus(state: State):
    from collections import Counter
    most_common = Counter(state["responses"]).most_common(1)[0][0]
    return {"final": most_common}

# 4. Build graph
builder = StateGraph(State)
builder.add_node("A", agent_a)
builder.add_node("B", agent_b)
builder.add_node("C", agent_c)
builder.add_node("Consensus", consensus)

builder.set_entry_point("A")
builder.add_edge("A", "B")
builder.add_edge("B", "C")
builder.add_edge("C", "Consensus")
builder.add_edge("Consensus", END)

graph = builder.compile()

# 5. Run
result = graph.invoke({"task": "What is the capital of France?", "responses": []})
print(result)


{'task': 'What is the capital of France?', 'responses': ['Paris is the capital of France.', 'The capital of France is Paris.', "France's capital city is Paris."], 'final': 'Paris is the capital of France.'}
